In [120]:

import yfinance as yf
import pandas as pd
import numpy as np
import logging
import alpaca_trade_api as tradeapi
from datetime import datetime


In [121]:
#If you want to trade on paper trading follow this additional step

API_KEY = 'PKUNA5TMMMBFZUM1BHLV'
SECRET_KEY = 'tKJaiBcC17Daq1WbjH2jZwKJ7w7QsLnnLASo24d5'
BASE_URL = 'https://paper-api.alpaca.markets'

# Create an instance of the Alpaca API
api = tradeapi.REST(API_KEY, SECRET_KEY, BASE_URL, api_version='v2')

# Download historical data
data = yf.download("EURJPY=X", start="2024-01-01", end="2024-12-01")
api.get_account()

[*********************100%***********************]  1 of 1 completed

Account({   'account_blocked': False,
    'account_number': 'PA3G9A6SI7R4',
    'accrued_fees': '0',
    'admin_configurations': {},
    'balance_asof': '2024-12-30',
    'bod_dtbp': '0',
    'buying_power': '200000',
    'cash': '100000',
    'created_at': '2024-12-19T12:25:35.726188Z',
    'crypto_status': 'ACTIVE',
    'crypto_tier': 1,
    'currency': 'USD',
    'daytrade_count': 0,
    'daytrading_buying_power': '0',
    'effective_buying_power': '200000',
    'equity': '100000',
    'id': 'cb6a9b69-d681-4cdf-acab-97f45727fa00',
    'initial_margin': '0',
    'intraday_adjustments': '0',
    'last_equity': '100000',
    'last_maintenance_margin': '0',
    'long_market_value': '0',
    'maintenance_margin': '0',
    'multiplier': '2',
    'non_marginable_buying_power': '100000',
    'options_approved_level': 2,
    'options_buying_power': '100000',
    'options_trading_level': 2,
    'pattern_day_trader': False,
    'pending_reg_taf_fees': '0',
    'portfolio_value': '100000',
    

In [122]:
###
def calculate_indicators(data):
    # Exponential Moving Averages
    data['EMA_12'] = data['Close'].ewm(span=12, adjust=False).mean()
    data['EMA_26'] = data['Close'].ewm(span=26, adjust=False).mean()
    # data['EMA_12'] = data['Close'].ewm(span=15, adjust=False).mean()
    # data['EMA_26'] = data['Close'].ewm(span=50, adjust=False).mean()
    # Relative Strength Index
    delta = data['Close'].diff()
    # gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    # loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    data['RSI'] = 100 - (100 / (1 + rs))
    return data

data = calculate_indicators(data)

In [124]:
def generate_signals(data):
    # Buy and Sell signals based on EMA and RSI
    pd.set_option('future.no_silent_downcasting', True)
    data['Signal'] = 0
    buy_signal = (data['EMA_12'] > data['EMA_26']) & (data['RSI'] < 30)
    sell_signal = (data['EMA_12'] < data['EMA_26']) & (data['RSI'] > 70)
    data.loc[buy_signal, 'Signal'] = 1
    data.loc[sell_signal, 'Signal'] = -1
    data['Position'] = data['Signal'].replace(to_replace=0, value=None).ffill()
    data['Signal'] = data['Signal'].fillna(0).astype(int)

    return data

In [127]:
generate_signals(data)

def backtest(data, initial_balance=10000):
    balance = initial_balance
    position = 0
    stop_loss = 0.95
    take_profit = 2.0
    entry_price = 0

    for i, row in data.iterrows():
        signal = row.Signal.iloc[0]
        close_price = row.Close.iloc[0]
        # print(type(row['Signal']), type(row['Close']))
        if signal  == 1 and balance > 0:
            position = balance / close_price
            balance = 0
            entry_price = close_price
            logging.info(f"BUY at {row['Close']} on {row.name.date()}")
        elif signal == -1 and position != 0:
            balance = position * close_price
            position = 0
            logging.info(f"SELL at {close_price} on {row.name.date()}")
            entry_price = 0

        if position != 0:
            if close_price <= entry_price * stop_loss:
                balance = position * close_price
                position = 0
                logging.info(f"STOP LOSS at {close_price} on {row.name.date()}")
            elif close_price >= entry_price * take_profit:
                balance = position * close_price
                position = 0
                logging.info(f"TAKE PROFIT at {close_price} on {row.name.date()}")

    if position != 0:
        balance = position * data.iloc[-1]['Close']  # Close position at the last available price

    return balance

In [128]:
logging.basicConfig(filename='trading_bot.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

final_balance = backtest(data)
print(final_balance)

Ticker
EURJPY=X    9799.894762
Name: 2024-11-29 00:00:00, dtype: object
